# 인터파크 베스트 셀러 도서 정보

In [ ]:
import requests
url = 'http://book.interpark.com/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028&smid1=s_menu&smid2=bestseller'
req = requests.get(url)
req.text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(req.text, 'html.parser')

### Step1. 찾고자 하는 목록들을 가져온다(list 계열의 li, table 계열의 tr)

In [3]:
# 화면 내의 모든 li 태그를 찾아줘서 이렇게 하면 안됨.
lis = soup.find_all('li')
len(lis)

3532

In [4]:
# 15개 베스트셀러 목록의 li 태그
lis = soup.select_one('.rankBestContentList').find_all('li')
len(lis)

58

In [5]:
# ol 태그의 자식 태그 li만 찾아주는 방법
lis = soup.select_one('.rankBestContentList').select('ol > li')
len(lis)

15

In [6]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

### Step 2. 목록들 중에 하나를 선택해 세부 내용을 가져온다.

In [7]:
li = lis[0]

In [8]:
# 책 제목
title = li.select_one('.itemName').get_text().strip()
title

'하얼빈'

In [9]:
# 저자
author = li.select_one('.author').get_text().strip()
author

'김훈(金薰)'

In [10]:
# 출판사
company = li.select_one('.company').get_text().strip()
company

'문학동네'

In [11]:
# 가격
price = li.select_one('.price > em').get_text().strip()
price = int(price.replace(',',''))
price

14400

In [12]:
# 순위
li = lis[13]
rank_spans = li.select('.rankBtn_ctrl')
rank_spans

[<span class="rankBtn_ctrl rkNum_M01 "></span>,
 <span class="rankBtn_ctrl rkNum_M02 "></span>]

In [13]:
for span in rank_spans:
  print(span['class'])

['rankBtn_ctrl', 'rkNum_M01', '']
['rankBtn_ctrl', 'rkNum_M02', '']


In [14]:
for span in rank_spans:
  print(span['class'][1][-1])

1
2


In [15]:
rank = ''
for span in rank_spans:
  rank += span['class'][1][-1]
rank = int(rank)
rank

12

### Step 3. 모든 목록에 대해서 세부 정보를 가져와서 데이터프레임으로 만든다

In [16]:
import pandas as pd

In [17]:
lines = []
for li in lis:
  title = li.select_one('.itemName').get_text().strip()
  author = li.select_one('.author').get_text().strip()
  company = li.select_one('.company').get_text().strip()
  price = li.select_one('.price > em').get_text().strip()
  price = int(price.replace(',',''))
  rank = ''
  rank_spans = li.select('.rankBtn_ctrl')
  for span in rank_spans:
    rank += span['class'][1][-1]
  rank = int(rank)
  lines.append([rank, title, author, company, price])

In [18]:
# 어디서 에러가 발생했는지 알고싶은 경우
lines = []
for i, li in enumerate(lis):
    try:
      title = li.select_one('.itemName').get_text().strip()
      author = li.select_one('.author').get_text().strip()
      company = li.select_one('.company').get_text().strip()
      price = li.select_one('.price > em').get_text().strip()
      price = int(price.replace(',',''))
      rank = ''
      rank_spans = li.select('.rankBtn_ctrl')
      for span in rank_spans:
        rank += span['class'][1][-1]
      rank = int(rank)
      lines.append([rank, title, author, company, price])
    except:
      print(i)

In [19]:
df = pd.DataFrame(lines, columns=['순위', '제목', '저자', '출판사', '가격'])
df

,순위,제목,저자,출판사,가격
0,1,하얼빈,김훈(金薰),문학동네,14400
1,2,미래의 부를 위한 투자 공부,신진상,미디어숲,16020
2,3,불편한 편의점 2,김호연,나무옆의자,12600
3,4,불편한 편의점(40만부 기념 벚꽃 에디션),김호연,나무옆의자,12600
4,5,계속 가보겠습니다,임은정,메디치미디어,16200
5,6,가불 선진국,조국,메디치미디어,14400
6,7,헤어질 결심 각본,"정서경, 박찬욱",을유문화사,13500
7,7,아직 긴 인생이 남았습니다,기시미 이치로(岸見一郞)/전경아 역,한국경제신문,14400
8,7,역행자,자청,웅진지식하우스,15750
9,10,파친코 1,이민진/신승미 역,인플루엔셜,14220
